In [45]:
import subprocess
import urllib.request
import gzip
from Bio import SeqIO
import os
import glob
import numpy as np

In [46]:
gzip.open("data/GCF_000022745.1_ASM2274v1.genomic.fna.gz", "rt")

<_io.TextIOWrapper name='data/GCF_000022745.1_ASM2274v1.genomic.fna.gz' encoding='UTF-8'>

In [47]:
jupyter notebook --NotebookApp.iopub_data_rate_limit = 1000000000

SyntaxError: invalid syntax (1194370450.py, line 1)

In [48]:
with gzip.open("data/GCF_000022745.1_ASM2274v1.genomic.fna.gz", "rt") as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        identifier = record.id
        description = record.description
        sequence = record.seq
#         print(sequence)
        print('Processing the record {}:'.format(identifier))
        print('Its description is: \n{}'.format(description))
        amount_of_nucleotides = len(sequence)
        print('Its sequence contains {} nucleotides.'.format(amount_of_nucleotides))

Processing the record NC_013119.1:
Its description is: 
NC_013119.1 Brucella microti CCM 4915 chromosome 1, complete sequence
Its sequence contains 2117050 nucleotides.
Processing the record NC_013118.1:
Its description is: 
NC_013118.1 Brucella microti CCM 4915 chromosome 2, complete sequence
Its sequence contains 1220319 nucleotides.


In [55]:
seq_list = list(sequence)
data = ''.join(seq_list)

In [50]:
class kmer_featurization:

  def __init__(self, k):
    """
    seqs: a list of DNA sequences
    k: the "k" in k-mer
    """
    self.k = k
    self.letters = ['A', 'T', 'C', 'G']
    self.multiplyBy = 4 ** np.arange(k-1, -1, -1) # the multiplying number for each digit position in the k-number system
    self.n = 4**k # number of possible k-mers

  def obtain_kmer_feature_for_a_list_of_sequences(self, seqs, write_number_of_occurrences=False):
    """
    Given a list of m DNA sequences, return a 2-d array with shape (m, 4**k) for the 1-hot representation of the kmer features.
    Args:
      write_number_of_occurrences:
        a boolean. If False, then in the 1-hot representation, the percentage of the occurrence of a kmer will be recorded; otherwise the number of occurrences will be recorded. Default False.    
    """
    kmer_features = []
    for seq in seqs:
      this_kmer_feature = self.obtain_kmer_feature_for_one_sequence(seq.upper(), write_number_of_occurrences=write_number_of_occurrences)
      kmer_features.append(this_kmer_feature)

    kmer_features = np.array(kmer_features)

    return kmer_features

  def obtain_kmer_feature_for_one_sequence(self, seq, write_number_of_occurrences=False):
    """
    Given a DNA sequence, return the 1-hot representation of its kmer feature.
    Args:
      seq: 
        a string, a DNA sequence
      write_number_of_occurrences:
        a boolean. If False, then in the 1-hot representation, the percentage of the occurrence of a kmer will be recorded; otherwise the number of occurrences will be recorded. Default False.
    """
    number_of_kmers = len(seq) - self.k + 1

    kmer_feature = np.zeros(self.n)

    for i in range(number_of_kmers):
      this_kmer = seq[i:(i+self.k)]
      this_numbering = self.kmer_numbering_for_one_kmer(this_kmer)
      kmer_feature[this_numbering] += 1

    if not write_number_of_occurrences:
      kmer_feature = kmer_feature / number_of_kmers

    return kmer_feature

  def kmer_numbering_for_one_kmer(self, kmer):
    """
    Given a k-mer, return its numbering (the 0-based position in 1-hot representation)
    """
    digits = []
    for letter in kmer:
      digits.append(self.letters.index(letter))

    digits = np.array(digits)

    numbering = (digits * self.multiplyBy).sum()

    return 

In [51]:
k = 4
obj = kmer_featurization(k) # initialize a kmer_featurization object
kmer_features = obj.obtain_kmer_feature_for_a_list_of_sequences(seq_list, write_number_of_occurrences=False)
# If you would like the k-mer features to be the percentage of occurrences (ranging from 0 to 1) as stated above, then leave write_number_of_occurrences as False (the default). If you prefer the features to be the counts for each k-mer occurrence, then set it to True.


# # If you just pass one sequence in string:
# seq = 'ATCGAGC'
# k = 6 
# obj = kmer_featurization(k) 
# kmer_feature = obj.obtain_kmer_feature_for_one_sequence(seq, write_number_of_occurrences=False)

In [52]:
kmer_features

array([[-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       ...,
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.]])

In [54]:
def count_kmers(read, k):
    """Count kmer occurrences in a given read.

    Parameters
    ----------
    read : string
        A single DNA sequence.
    k : int
        The value of k for which to count kmers.

    Returns
    -------
    counts : dictionary, {'string': int}
        A dictionary of counts keyed by their individual kmers (strings
        of length k).

    Examples
    --------
    >>> count_kmers("GATGAT", 3)
    {'ATG': 1, 'GAT': 2, 'TGA': 1}
    """
    # Start with an empty dictionary
    counts = {}
    # Calculate how many kmers of length k there are
    num_kmers = len(read) - k + 1
    # Loop over the kmer start positions
    for i in range(num_kmers):
        # Slice the string to get the kmer
        kmer = read[i:i+k]
        # Add the kmer to the dictionary if it's not there
        if kmer not in counts:
            counts[kmer] = 0
        # Increment the count for this kmer
        counts[kmer] += 1
    # Return the final counts
    return counts

In [56]:
count_kmers(data, 6)

{'GCGCAA': 1118,
 'CGCAAA': 597,
 'GCAAAA': 570,
 'CAAAAG': 419,
 'AAAAGA': 300,
 'AAAGAA': 316,
 'AAGAAA': 349,
 'AGAAAA': 374,
 'GAAAAA': 569,
 'AAAAAG': 389,
 'AAAAGG': 396,
 'AAAGGC': 641,
 'AAGGCC': 740,
 'AGGCCC': 415,
 'GGCCCC': 298,
 'GCCCCC': 251,
 'CCCCCA': 158,
 'CCCCAA': 158,
 'CCCAAA': 167,
 'CCAAAC': 188,
 'CAAACG': 223,
 'AAACGT': 133,
 'AACGTT': 106,
 'ACGTTG': 106,
 'CGTTGC': 426,
 'GTTGCC': 485,
 'TTGCCG': 1467,
 'TGCCGT': 651,
 'GCCGTC': 704,
 'CCGTCG': 484,
 'CGTCGT': 302,
 'GTCGTG': 333,
 'TCGTGG': 386,
 'CGTGGA': 401,
 'GTGGAA': 462,
 'TGGAAG': 586,
 'GGAAGC': 666,
 'GAAGCC': 878,
 'AAGCCC': 586,
 'AGCCCT': 291,
 'GCCCTT': 556,
 'CCCTTC': 368,
 'CCTTCT': 433,
 'CTTCTC': 328,
 'TTCTCA': 265,
 'TCTCAT': 222,
 'CTCATT': 218,
 'TCATTG': 412,
 'CATTGG': 470,
 'ATTGGT': 238,
 'TTGGTT': 166,
 'TGGTTT': 329,
 'GGTTTA': 104,
 'GTTTAG': 28,
 'TTTAGC': 54,
 'TTAGCA': 41,
 'TAGCAG': 65,
 'AGCAGC': 447,
 'GCAGCT': 319,
 'CAGCTA': 105,
 'AGCTAG': 23,
 'GCTAGA': 33,
 'CTAGAG': 2